In [1]:
from keras.models import load_model

import numpy as np
import librosa
import librosa.display
import sklearn
import matplotlib.pyplot as plt
import speech_recognition as VTT
from unicodedata import normalize
import os
import sys
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

In [3]:
VTE_Array = []
TTE_Array = []

In [4]:
CNN_model = load_model('weights/weight_result.h5')

In [5]:
for filename in os.listdir("WAV/"):
    filename = normalize('NFC', filename)
    try:
        if '.wav' not in filename in filename:
            continue
        wav, sr = librosa.load("WAV/" + filename, sr=None)
        print(filename)
        
        mfcc = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=100, n_fft=400, hop_length=160)
        mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
        padded_mfcc = pad2d(mfcc, 700)
        padded_mfcc = np.expand_dims(padded_mfcc, 0)
        
        VTE_result = CNN_model.predict(padded_mfcc)
        VTE_Array.append(VTE_result)
    
    except Exception as e:
        print(filename, e)
        raise

1.wav


c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


1/1 [==============================] - 0s 160ms/step
10.wav
1/1 [==============================] - 0s 34ms/step
2.wav
1/1 [==============================] - 0s 33ms/step


c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will

3.wav
1/1 [==============================] - 0s 35ms/step
4.wav


...

c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will

9.wav
1/1 [==============================] - 0s 33ms/step
Sample1.wav
1/1 [==============================] - ETA: 0s

c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will

1/1 [==============================] - 0s 34ms/step
Sample2.wav
1/1 [==============================] - 0s 32ms/step


c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


In [6]:
if not os.path.exists('TXT'):
    os.makedirs('TXT')

for filename in os.listdir("WAV/"):
    if not filename.endswith('.wav'):
        continue
    
    filename = normalize('NFC', filename)
    try:
        r = VTT.Recognizer()
        kr_audio = VTT.AudioFile("WAV/" + filename)
        with kr_audio as source:
            VTT_audio = r.record(source)
            
        temp = r.recognize_google(VTT_audio, language='ko-KR')
        
        # Get the file name without extension
        name = os.path.splitext(filename)[0]
        with open("TXT/" + str(name) + 'out.txt', 'w') as f:
            first_line = temp.split('\n')[0]
            print(first_line, file=f)
            
    except Exception as e:
        print(f"Error processing file {filename}: {str(e)}")

result2:
[]
Error processing file 1.wav: 
result2:
{   'alternative': [   {   'confidence': 0.73548317,
                           'transcript': '이게 무슨 소리야 소닉 하는데 다리가 왜 빨리 동지 보내라'},
                       {'transcript': '그게 무슨 소리야 소닉 하는데 다리가 왜 빨리 동지 보내라'},
                       {'transcript': '이게 무슨 소리야 소닉 하는데 다리가 왜 빨리 동지 내라'},
                       {'transcript': '이게 무슨 소리야 소닉 하는데 다리가 왜 빨리 동지 해라'},
                       {'transcript': '그게 무슨 소리야 소닉 하는데 다리가 왜 빨리 동지 내라'}],
    'final': True}
result2:
{   'alternative': [   {   'confidence': 0.77771437,
                           'transcript': '조금 그 자체가 아니라 소풍을 기다리는 마음에 있는 거'},
                       {'transcript': '소풍 그 자체가 아니라 소풍을 기다리는 마음에 있는 거'},
                       {'transcript': '적금 그 자체가 아니라 소풍을 기다리는 마음에 있는 거'},
                       {'transcript': '소금 그 자체가 아니라 소풍을 기다리는 마음에 있는 거'},
                       {'transcript': '저번 그 자체가 아니라 소풍을 기다리는 마음에 있는 거'}],
    'final': True}
result2:
{   'alternative': [   {   'confidence': 0

In [7]:
rnn_model = tf.keras.models.load_model("weights/model_04", custom_objects={"TextVectorization":TextVectorization})

In [8]:
for filename in os.listdir("TXT/"):
    filename = normalize('NFC', filename)
    try:
        if '.txt' not in filename in filename:
            continue
        f = open('TXT/' + filename)
        input_txt = f.read().strip()
        f.close()
        
        emotion = float(rnn_model.predict([input_txt]))
        TTE_Array.append(emotion)
        
    except Exception as e:
        print(filename, e)
        raise

1/1 [==============================] - 0s 30ms/step


In [9]:
TTE_Array

[0.5157862901687622,
 0.9937041997909546,
 0.2581300139427185,
 0.09714675694704056,
 0.2513967752456665,
 0.1110723540186882,
 0.39509114623069763,
 0.11372609436511993,
 0.3875333368778229,
 0.43533846735954285]

In [10]:
VTE_Array

[array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32),
 array([[0.5, 0.5]], dtype=float32)]